In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor

In [4]:
crab_df=pd.read_csv('train.csv',index_col=0)

In [5]:
crab_df

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
id,,,,,,,,,
0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8
...,...,...,...,...,...,...,...,...,...
74046,F,1.6625,1.2625,0.4375,50.660556,20.680960,10.361742,12.332033,10
74047,I,1.0750,0.8625,0.2750,10.446791,4.323299,2.296310,3.543687,6
74048,F,1.4875,1.2000,0.4125,29.483480,12.303683,7.540967,8.079607,10


In [6]:
crab_df=pd.get_dummies(crab_df,drop_first=True)

In [7]:
crab_df

,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age,Sex_I,Sex_M
id,,,,,,,,,,
0,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9,True,False
1,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8,True,False
2,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9,False,True
3,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11,False,False
4,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8,True,False
...,...,...,...,...,...,...,...,...,...,...
74046,1.6625,1.2625,0.4375,50.660556,20.680960,10.361742,12.332033,10,False,False
74047,1.0750,0.8625,0.2750,10.446791,4.323299,2.296310,3.543687,6,True,False
74048,1.4875,1.2000,0.4125,29.483480,12.303683,7.540967,8.079607,10,False,False


In [8]:
X=crab_df.drop('Age',axis=1)

In [9]:
y=crab_df['Age']

In [10]:
from sklearn.model_selection import StratifiedKFold

In [11]:
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=23)

In [12]:
from xgboost import XGBRegressor

In [13]:
rgb_regressor=XGBRegressor()

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
rgb_regressor.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': None,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [16]:
params={'learning_rate':np.linspace(0.001,0.999,5),'max_depth':[2,3,4,5],'n_estimators':[50,100,150]}

In [17]:
gcv=GridSearchCV(rgb_regressor,param_grid=params,cv=kfold,scoring='r2')

In [18]:
gcv.fit(X,y)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'learning_rate': array([0.001 , 0.2505, 0.5   , 0.7495, 0.999 ]),
                         'max_depth': [2, 3, 4, 5],
                         'n_estimators': [50, 100, 150]},
             scoring='r2')

In [19]:
gcv.best_params_

{'learning_rate': 0.2505, 'max_depth': 4, 'n_estimators': 100}

In [20]:
gcv.best_score_

0.584449226694348

In [22]:
from lightgbm import LGBMRegressor

In [23]:
lgbm_regressor=LGBMRegressor(random_state=23)

In [24]:
gcv=GridSearchCV(lgbm_regressor,param_grid=params,cv=kfold,scoring='r2')

In [25]:
gcv.fit(X,y)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000617 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1318
[LightGBM] [Info] Number of data points in the train set: 59240, number of used features: 9
[LightGBM] [Info] Start training from score 9.967387
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000655 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1319
[LightGBM] [Info] Number of data points in the train set: 59241, number of used features: 9
[LightGBM] [Info] Start training from score 9.967793
[LightGBM] [Warning] Found whi

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000664 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1320
[LightGBM] [Info] Number of data points in the train set: 59241, number of used features: 9
[LightGBM] [Info] Start training from score 9.968063
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000367 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1321
[LightGBM] [Info] Number of data points in the train set: 59241, number of used features: 9
[LightGBM] [Info] Start training from score 9.967691
[LightGBM] [Warning] Found whi

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000541 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1319
[LightGBM] [Info] Number of data points in the train set: 59241, number of used features: 9
[LightGBM] [Info] Start training from score 9.967793
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000341 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1322
[LightGBM] [Info] Number of data points in the train set: 59241, number of used features: 9
[LightGBM] [Info] Start training from score 9.968096
[LightGBM] [Warning] Found whi

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000422 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1319
[LightGBM] [Info] Number of data points in the train set: 59241, number of used features: 9
[LightGBM] [Info] Start training from score 9.967793
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000349 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1320
[LightGBM] [Info] Number of data points in the train set: 59241, number of used features: 9
[LightGBM] [Info] Start training from score 9.968063
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000358 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1321
[LightGBM] [Info] Number of data points in the train set: 59241, number of used features: 9
[LightGBM] [Info] Start training from score 9.967691
[LightGBM] [Warning] Found whi

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000290 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1320
[LightGBM] [Info] Number of data points in the train set: 59241, number of used features: 9
[LightGBM] [Info] Start training from score 9.968063
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001439 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1321
[LightGBM] [Info] Number of data points in the train set: 59241, number of used features: 9
[LightGBM] [Info] Start training from score 9.967691
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=LGBMRegressor(random_state=23),
             param_grid={'learning_rate': array([0.001 , 0.2505, 0.5   , 0.7495, 0.999 ]),
                         'max_depth': [2, 3, 4, 5],
                         'n_estimators': [50, 100, 150]},
             scoring='r2')

In [26]:
gcv.best_params_

{'learning_rate': 0.2505, 'max_depth': 3, 'n_estimators': 150}

In [27]:
gcv.best_score_

0.5854695590398074

## Cat_boost

In [28]:
from catboost import CatBoostRegressor

In [29]:
cgb_regeressor=CatBoostRegressor(random_state=23)

In [30]:
gcv=GridSearchCV(cgb_regeressor,param_grid=params,cv=kfold,scoring='r2')

In [31]:
gcv.fit(X,y)

0:	learn: 3.1733916	total: 53.1ms	remaining: 2.6s
1:	learn: 3.1722097	total: 59.2ms	remaining: 1.42s
2:	learn: 3.1710305	total: 63.3ms	remaining: 992ms
3:	learn: 3.1698424	total: 67.9ms	remaining: 781ms
4:	learn: 3.1686592	total: 71.4ms	remaining: 642ms
5:	learn: 3.1674747	total: 75.1ms	remaining: 551ms
6:	learn: 3.1663044	total: 78.7ms	remaining: 483ms
7:	learn: 3.1651215	total: 81.8ms	remaining: 429ms
8:	learn: 3.1639564	total: 84.6ms	remaining: 385ms
9:	learn: 3.1627910	total: 87.4ms	remaining: 350ms
10:	learn: 3.1616175	total: 90.3ms	remaining: 320ms
11:	learn: 3.1604485	total: 92.8ms	remaining: 294ms
12:	learn: 3.1592793	total: 95.4ms	remaining: 272ms
13:	learn: 3.1581243	total: 97.9ms	remaining: 252ms
14:	learn: 3.1569569	total: 101ms	remaining: 235ms
15:	learn: 3.1557999	total: 103ms	remaining: 220ms
16:	learn: 3.1546252	total: 106ms	remaining: 205ms
17:	learn: 3.1534597	total: 108ms	remaining: 192ms
18:	learn: 3.1523099	total: 110ms	remaining: 179ms
19:	learn: 3.1512058	total: 

0:	learn: 3.1738879	total: 5.42ms	remaining: 266ms
1:	learn: 3.1727079	total: 10.5ms	remaining: 253ms
2:	learn: 3.1715242	total: 17.3ms	remaining: 271ms
3:	learn: 3.1703375	total: 21.9ms	remaining: 251ms
4:	learn: 3.1691562	total: 25.2ms	remaining: 227ms
5:	learn: 3.1679807	total: 28.9ms	remaining: 212ms
6:	learn: 3.1668120	total: 32.4ms	remaining: 199ms
7:	learn: 3.1656306	total: 35.5ms	remaining: 186ms
8:	learn: 3.1644704	total: 38.7ms	remaining: 176ms
9:	learn: 3.1633417	total: 42ms	remaining: 168ms
10:	learn: 3.1621694	total: 44.6ms	remaining: 158ms
11:	learn: 3.1610001	total: 48.2ms	remaining: 153ms
12:	learn: 3.1598437	total: 50.8ms	remaining: 145ms
13:	learn: 3.1587077	total: 53.5ms	remaining: 137ms
14:	learn: 3.1575490	total: 55.7ms	remaining: 130ms
15:	learn: 3.1563924	total: 57.8ms	remaining: 123ms
16:	learn: 3.1552855	total: 60.1ms	remaining: 117ms
17:	learn: 3.1541546	total: 62.1ms	remaining: 110ms
18:	learn: 3.1530259	total: 64.4ms	remaining: 105ms
19:	learn: 3.1519126	tot

0:	learn: 3.1743854	total: 6.21ms	remaining: 614ms
1:	learn: 3.1732135	total: 11.3ms	remaining: 555ms
2:	learn: 3.1720369	total: 15.9ms	remaining: 515ms
3:	learn: 3.1708540	total: 19.8ms	remaining: 476ms
4:	learn: 3.1696718	total: 23.8ms	remaining: 452ms
5:	learn: 3.1684955	total: 27ms	remaining: 424ms
6:	learn: 3.1673316	total: 30.1ms	remaining: 400ms
7:	learn: 3.1661542	total: 33.4ms	remaining: 384ms
8:	learn: 3.1649917	total: 36.5ms	remaining: 369ms
9:	learn: 3.1638714	total: 39.3ms	remaining: 353ms
10:	learn: 3.1627628	total: 41.7ms	remaining: 338ms
11:	learn: 3.1615955	total: 44.1ms	remaining: 323ms
12:	learn: 3.1604290	total: 47.6ms	remaining: 318ms
13:	learn: 3.1592765	total: 49.9ms	remaining: 307ms
14:	learn: 3.1581730	total: 52.4ms	remaining: 297ms
15:	learn: 3.1570796	total: 54.8ms	remaining: 288ms
16:	learn: 3.1559190	total: 57ms	remaining: 278ms
17:	learn: 3.1547459	total: 59ms	remaining: 269ms
18:	learn: 3.1536252	total: 61.2ms	remaining: 261ms
19:	learn: 3.1524846	total: 

95:	learn: 3.0693969	total: 200ms	remaining: 8.34ms
96:	learn: 3.0683629	total: 202ms	remaining: 6.25ms
97:	learn: 3.0673527	total: 204ms	remaining: 4.16ms
98:	learn: 3.0663354	total: 206ms	remaining: 2.08ms
99:	learn: 3.0653089	total: 207ms	remaining: 0us
0:	learn: 3.1738879	total: 5.19ms	remaining: 514ms
1:	learn: 3.1727079	total: 9.97ms	remaining: 489ms
2:	learn: 3.1715242	total: 15.1ms	remaining: 488ms
3:	learn: 3.1703375	total: 19.3ms	remaining: 463ms
4:	learn: 3.1691562	total: 23.3ms	remaining: 443ms
5:	learn: 3.1679807	total: 26.7ms	remaining: 419ms
6:	learn: 3.1668120	total: 30.4ms	remaining: 405ms
7:	learn: 3.1656306	total: 34.3ms	remaining: 395ms
8:	learn: 3.1644704	total: 37.7ms	remaining: 381ms
9:	learn: 3.1633417	total: 40.6ms	remaining: 366ms
10:	learn: 3.1621694	total: 43.5ms	remaining: 352ms
11:	learn: 3.1610001	total: 46.4ms	remaining: 340ms
12:	learn: 3.1598437	total: 49.1ms	remaining: 328ms
13:	learn: 3.1587077	total: 52ms	remaining: 319ms
14:	learn: 3.1575490	total:

96:	learn: 3.0678363	total: 201ms	remaining: 110ms
97:	learn: 3.0668466	total: 203ms	remaining: 108ms
98:	learn: 3.0658224	total: 205ms	remaining: 105ms
99:	learn: 3.0648192	total: 206ms	remaining: 103ms
100:	learn: 3.0637883	total: 208ms	remaining: 101ms
101:	learn: 3.0627713	total: 210ms	remaining: 98.6ms
102:	learn: 3.0617783	total: 211ms	remaining: 96.4ms
103:	learn: 3.0607595	total: 213ms	remaining: 94.1ms
104:	learn: 3.0597254	total: 215ms	remaining: 92ms
105:	learn: 3.0587477	total: 217ms	remaining: 89.9ms
106:	learn: 3.0577533	total: 218ms	remaining: 87.8ms
107:	learn: 3.0567046	total: 220ms	remaining: 85.6ms
108:	learn: 3.0557107	total: 222ms	remaining: 83.5ms
109:	learn: 3.0547270	total: 224ms	remaining: 81.4ms
110:	learn: 3.0536835	total: 226ms	remaining: 79.2ms
111:	learn: 3.0526904	total: 227ms	remaining: 77.1ms
112:	learn: 3.0516681	total: 229ms	remaining: 74.9ms
113:	learn: 3.0506352	total: 231ms	remaining: 72.8ms
114:	learn: 3.0496566	total: 233ms	remaining: 70.9ms
115:

...

67:	learn: 1.9803520	total: 203ms	remaining: 245ms
68:	learn: 1.9792423	total: 206ms	remaining: 242ms
69:	learn: 1.9778449	total: 209ms	remaining: 238ms
70:	learn: 1.9766011	total: 211ms	remaining: 235ms
71:	learn: 1.9752083	total: 214ms	remaining: 231ms
72:	learn: 1.9741530	total: 216ms	remaining: 228ms
73:	learn: 1.9722373	total: 219ms	remaining: 224ms
74:	learn: 1.9711968	total: 221ms	remaining: 221ms
75:	learn: 1.9700683	total: 223ms	remaining: 218ms
76:	learn: 1.9688130	total: 226ms	remaining: 214ms
77:	learn: 1.9677185	total: 229ms	remaining: 211ms
78:	learn: 1.9664496	total: 231ms	remaining: 207ms
79:	learn: 1.9660728	total: 233ms	remaining: 204ms
80:	learn: 1.9650819	total: 236ms	remaining: 201ms
81:	learn: 1.9645702	total: 238ms	remaining: 197ms
82:	learn: 1.9630647	total: 240ms	remaining: 194ms
83:	learn: 1.9620213	total: 243ms	remaining: 191ms
84:	learn: 1.9613221	total: 245ms	remaining: 187ms
85:	learn: 1.9600977	total: 247ms	remaining: 184ms
86:	learn: 1.9594155	total: 250

147:	learn: 1.8934699	total: 402ms	remaining: 5.43ms
148:	learn: 1.8926927	total: 405ms	remaining: 2.71ms
149:	learn: 1.8920165	total: 407ms	remaining: 0us
0:	learn: 2.3739376	total: 7.09ms	remaining: 1.06s
1:	learn: 2.2308160	total: 14.2ms	remaining: 1.05s
2:	learn: 2.1935917	total: 19.9ms	remaining: 975ms
3:	learn: 2.1676394	total: 24.7ms	remaining: 902ms
4:	learn: 2.1442889	total: 30.2ms	remaining: 875ms
5:	learn: 2.1304022	total: 34.4ms	remaining: 826ms
6:	learn: 2.1216327	total: 38.5ms	remaining: 787ms
7:	learn: 2.1112831	total: 42.5ms	remaining: 755ms
8:	learn: 2.1047239	total: 46.4ms	remaining: 727ms
9:	learn: 2.1003078	total: 50ms	remaining: 700ms
10:	learn: 2.0930541	total: 53.1ms	remaining: 671ms
11:	learn: 2.0907140	total: 56ms	remaining: 645ms
12:	learn: 2.0853498	total: 58.9ms	remaining: 621ms
13:	learn: 2.0822295	total: 61.8ms	remaining: 600ms
14:	learn: 2.0808790	total: 64.2ms	remaining: 578ms
15:	learn: 2.0750037	total: 67ms	remaining: 561ms
16:	learn: 2.0698026	total: 

68:	learn: 1.9617883	total: 205ms	remaining: 240ms
69:	learn: 1.9605371	total: 208ms	remaining: 237ms
70:	learn: 1.9596075	total: 210ms	remaining: 234ms
71:	learn: 1.9590866	total: 212ms	remaining: 230ms
72:	learn: 1.9579358	total: 215ms	remaining: 227ms
73:	learn: 1.9561523	total: 218ms	remaining: 224ms
74:	learn: 1.9555969	total: 220ms	remaining: 220ms
75:	learn: 1.9541982	total: 223ms	remaining: 217ms
76:	learn: 1.9529123	total: 225ms	remaining: 214ms
77:	learn: 1.9514334	total: 228ms	remaining: 210ms
78:	learn: 1.9510664	total: 230ms	remaining: 207ms
79:	learn: 1.9497990	total: 233ms	remaining: 204ms
80:	learn: 1.9487878	total: 235ms	remaining: 200ms
81:	learn: 1.9471121	total: 238ms	remaining: 197ms
82:	learn: 1.9463388	total: 240ms	remaining: 194ms
83:	learn: 1.9456855	total: 243ms	remaining: 191ms
84:	learn: 1.9443058	total: 245ms	remaining: 187ms
85:	learn: 1.9429032	total: 247ms	remaining: 184ms
86:	learn: 1.9416300	total: 250ms	remaining: 181ms
87:	learn: 1.9406960	total: 252

149:	learn: 1.8996101	total: 400ms	remaining: 0us
0:	learn: 2.8478245	total: 8.63ms	remaining: 1.28s
1:	learn: 2.6252961	total: 16ms	remaining: 1.19s
2:	learn: 2.4740284	total: 22.6ms	remaining: 1.1s
3:	learn: 2.3739108	total: 28ms	remaining: 1.02s
4:	learn: 2.3079088	total: 33.3ms	remaining: 966ms
5:	learn: 2.2592570	total: 38ms	remaining: 911ms
6:	learn: 2.2251640	total: 42.6ms	remaining: 870ms
7:	learn: 2.2002274	total: 46.7ms	remaining: 829ms
8:	learn: 2.1801415	total: 50.4ms	remaining: 790ms
9:	learn: 2.1678310	total: 53.9ms	remaining: 755ms
10:	learn: 2.1564055	total: 57.2ms	remaining: 723ms
11:	learn: 2.1497347	total: 60.4ms	remaining: 694ms
12:	learn: 2.1371677	total: 63.8ms	remaining: 673ms
13:	learn: 2.1275125	total: 66.6ms	remaining: 647ms
14:	learn: 2.1235334	total: 69.2ms	remaining: 623ms
15:	learn: 2.1185686	total: 71.9ms	remaining: 602ms
16:	learn: 2.1122632	total: 74.7ms	remaining: 584ms
17:	learn: 2.1063765	total: 77.6ms	remaining: 569ms
18:	learn: 2.1011015	total: 80.

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=23, shuffle=True),
             estimator=<catboost.core.CatBoostRegressor object at 0x7f63c4064fd0>,
             param_grid={'learning_rate': array([0.001 , 0.2505, 0.5   , 0.7495, 0.999 ]),
                         'max_depth': [2, 3, 4, 5],
                         'n_estimators': [50, 100, 150]},
             scoring='r2')

In [32]:
gvc.best_params_

NameError: name 'gvc' is not defined

In [ ]:
gcv.best_score_